In [1]:
import logging
import numpy as np
import os
import torch
import torchvision

from pathlib import Path
from pytorch_lightning import seed_everything
from torchvision import transforms

SELECTED_CONCEPTS = [
    2,
    4,
    6,
    7,
    8,
    9,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    32,
    33,
    39,
]

CONCEPT_SEMANTICS = [
    '5_o_Clock_Shadow',
    'Arched_Eyebrows',
    'Attractive',
    'Bags_Under_Eyes',
    'Bald',
    'Bangs',
    'Big_Lips',
    'Big_Nose',
    'Black_Hair',
    'Blond_Hair',
    'Blurry',
    'Brown_Hair',
    'Bushy_Eyebrows',
    'Chubby',
    'Double_Chin',
    'Eyeglasses',
    'Goatee',
    'Gray_Hair',
    'Heavy_Makeup',
    'High_Cheekbones',
    'Male',
    'Mouth_Slightly_Open',
    'Mustache',
    'Narrow_Eyes',
    'No_Beard',
    'Oval_Face',
    'Pale_Skin',
    'Pointy_Nose',
    'Receding_Hairline',
    'Rosy_Cheeks',
    'Sideburns',
    'Smiling',
    'Straight_Hair',
    'Wavy_Hair',
    'Wearing_Earrings',
    'Wearing_Hat',
    'Wearing_Lipstick',
    'Wearing_Necklace',
    'Wearing_Necktie',
    'Young',
]

/home/disk/qij/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


# Generate data
`config`
`seed`
`output_data_var`

In [ ]:
root_dir = '../../celeba'
concept_group_map = None
seed_everything(42)
use_binary_vector_class = True
label_binary_width = 1
width = 1

def _binarize(concepts, selected, width):
    result = []
    binary_repr = []
    concepts = concepts[selected]
    for i in range(0, concepts.shape[-1], width):
        binary_repr.append(
            str(int(np.sum(concepts[i : i + width]) > 0))
        )
    return int("".join(binary_repr), 2)




Seed set to 42


In [38]:
import gdown
celeba_train_data = torchvision.datasets.CelebA(
            root=root_dir,
            split='all',
            download=True,
            target_transform=lambda x: x[0].long() - 1,
            # target_type=['attr'],
        )


Files already downloaded and verified


In [42]:
celeba_train_data.target_transform = lambda x: x#@x[0].long() - 1
celeba_train_data.target_type = ['attr']
print(celeba_train_data.attr)
print(celeba_train_data.extra_repr)
print(celeba_train_data[0])

tensor([[0, 1, 1,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 1, 1,  ..., 0, 0, 1],
        [0, 1, 1,  ..., 0, 0, 1]])
<bound method CelebA.extra_repr of Dataset CelebA
    Number of datapoints: 202599
    Root location: ../../celeba
    Target type: ['attr']
    Split: all
    StandardTransform
Target transform: <function <lambda> at 0x7ea838176a20>>
(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7EA83816EE10>, tensor([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1]))


In [29]:
concept_freq = np.sum(
    celeba_train_data.attr.cpu().detach().numpy(),
    axis=0
) / celeba_train_data.attr.shape[0]
print(f"Concept frequency is: {concept_freq}")
sorted_concepts = list(map(
    lambda x: x[0],
    sorted(enumerate(np.abs(concept_freq - 0.5)), key=lambda x: x[1]),
))
print(sorted_concepts, '\n', concept_freq[sorted_concepts])
num_concepts = 6
concept_idxs = sorted_concepts[:num_concepts]
concept_idxs = sorted(concept_idxs)
print(concept_idxs)

Concept frequency is: [0.11113579 0.26698059 0.512505   0.20457159 0.02244335 0.15157528
 0.24079586 0.23453225 0.23925093 0.14799185 0.05089857 0.20519351
 0.14216753 0.05756692 0.04668829 0.06511878 0.06276438 0.04194986
 0.38692195 0.45503186 0.41675428 0.48342786 0.04154512 0.11514864
 0.83493996 0.28414257 0.0429469  0.27744461 0.07977828 0.06572096
 0.05651064 0.48208037 0.20840182 0.31956722 0.18892492 0.04846026
 0.4724357  0.12296704 0.07271507 0.77361685]
[2, 21, 31, 36, 19, 20, 18, 33, 25, 27, 1, 6, 8, 7, 39, 32, 11, 3, 34, 24, 5, 9, 12, 37, 23, 0, 28, 38, 29, 15, 16, 13, 30, 10, 35, 14, 26, 17, 22, 4] 
 [0.512505   0.48342786 0.48208037 0.4724357  0.45503186 0.41675428
 0.38692195 0.31956722 0.28414257 0.27744461 0.26698059 0.24079586
 0.23925093 0.23453225 0.77361685 0.20840182 0.20519351 0.20457159
 0.18892492 0.83493996 0.15157528 0.14799185 0.14216753 0.12296704
 0.11514864 0.11113579 0.07977828 0.07271507 0.06572096 0.06511878
 0.06276438 0.05756692 0.05651064 0.050898

In [30]:
num_hidden = 2
hidden_concepts = sorted(
                sorted_concepts[
                    num_concepts:min(
                        (num_concepts + num_hidden),
                        len(sorted_concepts)
                    )
                ]
            )
print(hidden_concepts)

[18, 33]


In [ ]:
image_size = 64
print(f"Selecting concepts: {concept_idxs}")
print(f"\tAnd hidden concepts: {hidden_concepts}")
celeba_train_data = torchvision.datasets.CelebA(
    root=root_dir,
    split='all',
    download=True,
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.ConvertImageDtype(torch.float32),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]),
    target_transform=lambda x: [
        torch.tensor(
            _binarize(
                x[1].cpu().detach().numpy(),
                selected=(concept_idxs + hidden_concepts),
                width=width,
            ),
            dtype=torch.long,
        ),
        x[1][concept_idxs].float(),
    ],
    target_type=['identity', 'attr'],
)
print(celeba_train_data.attr.shape, celeba_train_data.identity.shape)
print(celeba_train_data[0]) # (image, [class, concept])

Selecting concepts: [2, 19, 20, 21, 31, 36]
	And hidden concepts: [18, 33]
Files already downloaded and verified
torch.Size([202599, 40]) torch.Size([202599, 1])
(tensor([[[ 1.0000,  1.0000,  1.0000,  ...,  0.9451,  0.9529,  0.9529],
         [ 0.9922,  0.9922,  0.9922,  ...,  0.9451,  0.9529,  0.9451],
         [ 0.9922,  0.9922,  0.9922,  ...,  0.9294,  0.9608,  0.9529],
         ...,
         [ 0.3020,  0.2627,  0.1137,  ...,  0.4196,  0.4510,  0.4510],
         [ 0.3569,  0.2941,  0.2627,  ...,  0.3725,  0.3490,  0.3647],
         [ 0.3882,  0.3647,  0.3804,  ...,  0.5608,  0.4824,  0.4196]],

        [[ 0.8431,  0.8431,  0.8431,  ...,  0.8667,  0.8588,  0.8902],
         [ 0.8431,  0.8431,  0.8431,  ...,  0.8824,  0.8667,  0.8980],
         [ 0.8588,  0.8588,  0.8588,  ...,  0.8902,  0.8745,  0.8902],
         ...,
         [-0.2078, -0.2235, -0.3255,  ...,  0.0275,  0.0510,  0.0510],
         [-0.1529, -0.2078, -0.2314,  ..., -0.0667, -0.0667, -0.0353],
         [-0.1294, -0.1373

In [32]:
label_remap = {}
vals, counts = np.unique(
    list(map(
        lambda x: _binarize(
            x.cpu().detach().numpy(),
            selected=(concept_idxs + hidden_concepts),
            width=width,
        ),
        celeba_train_data.attr
    )),
    return_counts=True,
)

print(vals, counts)
for i, label in enumerate(vals):
    label_remap[label] = i

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  44  45  46  48  49  50  52  53  54  56  57  59
  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77
  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95
  96  97  99 100 102 104 105 106 108 109 110 111 112 113 114 116 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157
 158 159 160 161 162 163 164 165 166 167 168 169 170 172 173 174 175 176
 177 178 180 181 182 183 184 185 186 188 189 190 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 228 230 231 232 233 234 236
 237 238 239 240 241 244 248 249 250 251 252 253 254 255] [ 4552  1292   288    85  1581  1083  1811  1281   237    60     9

In [33]:
print(label_remap)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 44: 42, 45: 43, 46: 44, 48: 45, 49: 46, 50: 47, 52: 48, 53: 49, 54: 50, 56: 51, 57: 52, 59: 53, 60: 54, 61: 55, 62: 56, 63: 57, 64: 58, 65: 59, 66: 60, 67: 61, 68: 62, 69: 63, 70: 64, 71: 65, 72: 66, 73: 67, 74: 68, 75: 69, 76: 70, 77: 71, 78: 72, 79: 73, 80: 74, 81: 75, 82: 76, 83: 77, 84: 78, 85: 79, 86: 80, 87: 81, 88: 82, 89: 83, 90: 84, 91: 85, 92: 86, 93: 87, 94: 88, 95: 89, 96: 90, 97: 91, 99: 92, 100: 93, 102: 94, 104: 95, 105: 96, 106: 97, 108: 98, 109: 99, 110: 100, 111: 101, 112: 102, 113: 103, 114: 104, 116: 105, 120: 106, 121: 107, 122: 108, 123: 109, 124: 110, 125: 111, 126: 112, 127: 113, 128: 114, 129: 115, 130: 116, 131: 117, 132: 118, 133: 119, 134: 120, 13

In [34]:
celeba_train_data = torchvision.datasets.CelebA(
    root=root_dir,
    split='all',
    download=True,
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.ConvertImageDtype(torch.float32),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]),
    target_transform=lambda x: [
        torch.tensor(
            label_remap[_binarize(
                x[1].cpu().detach().numpy(),
                selected=(concept_idxs + hidden_concepts),
                width=width,
            )],
            dtype=torch.long,
        ),
        x[1][concept_idxs].float(),
    ],
    target_type=['identity', 'attr'],
)
num_classes = len(label_remap)
print(num_classes)
print(celeba_train_data.attr.shape, celeba_train_data.identity.shape)
print(celeba_train_data[0]) # (image, [class, concept])

Files already downloaded and verified
230
torch.Size([202599, 40]) torch.Size([202599, 1])
(tensor([[[ 1.0000,  1.0000,  1.0000,  ...,  0.9451,  0.9529,  0.9529],
         [ 0.9922,  0.9922,  0.9922,  ...,  0.9451,  0.9529,  0.9451],
         [ 0.9922,  0.9922,  0.9922,  ...,  0.9294,  0.9608,  0.9529],
         ...,
         [ 0.3020,  0.2627,  0.1137,  ...,  0.4196,  0.4510,  0.4510],
         [ 0.3569,  0.2941,  0.2627,  ...,  0.3725,  0.3490,  0.3647],
         [ 0.3882,  0.3647,  0.3804,  ...,  0.5608,  0.4824,  0.4196]],

        [[ 0.8431,  0.8431,  0.8431,  ...,  0.8667,  0.8588,  0.8902],
         [ 0.8431,  0.8431,  0.8431,  ...,  0.8824,  0.8667,  0.8980],
         [ 0.8588,  0.8588,  0.8588,  ...,  0.8902,  0.8745,  0.8902],
         ...,
         [-0.2078, -0.2235, -0.3255,  ...,  0.0275,  0.0510,  0.0510],
         [-0.1529, -0.2078, -0.2314,  ..., -0.0667, -0.0667, -0.0353],
         [-0.1294, -0.1373, -0.1059,  ...,  0.1451,  0.0824,  0.0275]],

        [[ 0.5529,  0.55

In [35]:
label_dataset_subsample = 12
# subsample the dataset
factor = label_dataset_subsample
if factor != 1:
    train_idxs = np.random.choice(
        np.arange(0, len(celeba_train_data)),
        replace=False,
        size=len(celeba_train_data)//factor,
    )
    logging.debug(f"Subsampling to {len(train_idxs)} elements.")
    celeba_train_data = torch.utils.data.Subset(
        celeba_train_data,
        train_idxs,
    )
    print(celeba_train_data.__len__())

16883


In [ ]:
BATCH_SIZE = 64
NUM_WORKERS = 4
total_samples = len(celeba_train_data)
train_samples = int(0.7 * total_samples)
test_samples = int(0.2 * total_samples)
val_samples = total_samples - test_samples - train_samples
logging.debug(
    f"Data split is: {total_samples} = {train_samples} (train) + "
    f"{test_samples} (test) + {val_samples} (validation)"
)
celeba_train_data, celeba_test_data, celeba_val_data = \
    torch.utils.data.random_split(
        celeba_train_data,
        [train_samples, test_samples, val_samples],
    )
train_dl = torch.utils.data.DataLoader(
    celeba_train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
)
test_dl = torch.utils.data.DataLoader(
    celeba_test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
)
val_dl = torch.utils.data.DataLoader(
    celeba_val_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
)

当你使用 `torchvision.datasets.CelebA` 下载 CelebA 数据集时，数据集的文件格式和结构如下所示：

### 下载后的文件结构：
下载后，CelebA 数据集将被存储在指定的 `root` 目录下，通常包括以下内容：

```
root/
    CelebA/
        img_align_celeba/        # 存放图片的文件夹
        list_attr_celeba.txt      # 属性标签文件
        list_eval_partition.txt   # 数据集划分文件
        list_identity_celeba.txt  # 人物身份标签文件
        list_landmarks_celeba.txt # 关键点标签文件
```

#### 各个文件说明：
1. **`img_align_celeba/`**  
   这个文件夹包含了所有的 CelebA 图片。图片的命名格式通常是以 `000001.jpg`, `000002.jpg` 这样的数字作为文件名。每张图片的尺寸为 `178x218`，它们通常会经过对齐和裁剪。

2. **`list_attr_celeba.txt`**  
   该文件包含了每张图片的属性标签，每行对应一张图片的多个二进制属性（例如：是否微笑、是否戴眼镜、是否穿着耳环等）。文件内容通常是这样的：
   ```
   000001.jpg 1 -1 1 -1 1 1 1 -1 1 0 ...  # 对应于图片000001.jpg的多个属性
   000002.jpg -1 1 -1 1 -1 1 1 1 -1 1 ...  # 对应于图片000002.jpg的多个属性
   ```

3. **`list_eval_partition.txt`**  
   该文件包含了数据集的划分信息，用于训练、验证和测试。它指示每张图片属于哪个数据集（训练集、验证集或测试集）。文件内容通常是这样的：
   ```
   000001.jpg 0
   000002.jpg 1
   000003.jpg 2
   ...
   ```
   其中，`0` 表示训练集，`1` 表示验证集，`2` 表示测试集。

4. **`list_identity_celeba.txt`**  
   该文件包含每张图片的人物身份标签。每个标签是一个整数，表示该图片属于哪个身份（人物）。内容类似：
   ```
   000001.jpg 1
   000002.jpg 2
   000003.jpg 1
   ...
   ```
   其中，`1`, `2` 等是人物的ID编号。

5. **`list_landmarks_celeba.txt`**  
   该文件包含每张图片的五个关键点的坐标（如眼睛、鼻子、嘴巴的位置）。内容可能类似于：
   ```
   000001.jpg x1 y1 x2 y2 x3 y3 x4 y4 x5 y5
   000002.jpg x1 y1 x2 y2 x3 y3 x4 y4 x5 y5
   ...
   ```
   每行表示一张图片的五个面部关键点的坐标。

### 读取和使用：
- **图片**：你可以通过 `torchvision.datasets.CelebA` 直接加载图片（例如通过 `transform` 转换为 Tensor）。
- **属性标签**：你可以通过 `attr` 参数来获取每张图片的属性标签，例如是否微笑、是否戴眼镜等。
- **数据集划分**：你可以使用 `split` 参数来加载不同的数据集部分（训练集、验证集、测试集）。
- **关键点信息**：如果需要，可以使用 `landmarks` 参数来加载每张图片的五个关键点坐标。

例如：

```python
from torchvision import datasets, transforms

data_dir = '/path/to/CelebA'
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# 加载训练集
celeba_dataset = datasets.CelebA(root=data_dir, split='train', transform=transform, download=False)

# 查看第一个样本
image, label = celeba_dataset[0]
print(image.shape)  # 图片大小 (3, 128, 128)
print(label)        # 属性标签
```

### 小结：
下载后的 CelebA 数据集包含了图片、属性标签、身份标签、数据集划分信息等。你可以根据需要，选择性地加载和处理这些文件。如果你需要处理属性标签、身份标签或关键点信息，可以从相应的文本文件中读取。